In [2]:
"""
Created on Sun Apr 19 22:17:19 2020

@author: Jonas
"""
#Soft Baseline
import fasttext

#General
import pandas as pd

#Hard Baseline library

from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn import model_selection, svm
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.metrics import classification_report

#NNModel library
from keras.models import Sequential
from keras import layers
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.layers import Dropout

Using TensorFlow backend.


In [3]:
#Load already trained fasttext model an predit

#model = fasttext.train_supervised(r"C:\Users\Jonas\Documents\GSOM\NLP\train.ft.txt",lr=1.0, epoch=3, wordNgrams=2)
#model.save_model("model_fasttext_classification.bin")  

model=fasttext.load_model("model_fasttext_classification.bin")

In [4]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

print_results(*model.test(r"C:\Users\Jonas\Documents\GSOM\NLP\test.ft.txt"))  

N	400000
P@1	0.938
R@1	0.938


In [3]:

test = pd.read_csv(r"C:\Users\Jonas\Documents\GSOM\NLP\test.ft.txt",sep="\n",header=None)  
train = pd.read_csv(r"C:\Users\Jonas\Documents\GSOM\NLP\train.ft.txt",sep="\n",header=None)


test.columns=['text']
train.columns=['text']

In [4]:

#Make clean Train Dataframe with 2 Columns
train_clean= train.copy()
train_clean['text']=train_clean['text'].str.replace("__label__1","1 \t")
train_clean['text']=train_clean['text'].str.replace("__label__2","2 \t")
train_clean['label'],train_clean['text'] = train_clean['text'].str.split('\t').str

print(train_clean.groupby('label').size())

C:\Users\Jonas\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """


label
1     1800000
2     1800000
dtype: int64


In [5]:

#Make clean Test Dataframe with 2 Columns
test_clean= test.copy()
test_clean['text']=test_clean['text'].str.replace("__label__1","1 \t")
test_clean['text']=test_clean['text'].str.replace("__label__2","2 \t")
test_clean['label'],test_clean['text'] = test_clean['text'].str.split('\t').str

print(test_clean.groupby('label').size())

C:\Users\Jonas\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: Columnar iteration over characters will be deprecated in future releases.
  """


label
1     200000
2     200000
dtype: int64


In [6]:
print(train_clean.head())
print(test_clean.head())

                                                text label
0   Stuning even for the non-gamer: This sound tr...    2 
1   The best soundtrack ever to anything.: I'm re...    2 
2   Amazing!: This soundtrack is my favorite musi...    2 
3   Excellent Soundtrack: I truly like this sound...    2 
4   Remember, Pull Your Jaw Off The Floor After H...    2 
                                                text label
0   Great CD: My lovely Pat has one of the GREAT ...    2 
1   One of the best game music soundtracks - for ...    2 
2   Batteries died within a year ...: I bought th...    1 
3   works fine, but Maha Energy is better: Check ...    2 
4   Great for the non-audiophile: Reviewed quite ...    2 


In [7]:
#Using sklearn vectorizer to create the word matrix and dictornary and transform the test data to it. I thought top 50k words will be enough, otherwise the model (espeically normal SVM takes for ever)
cv = CountVectorizer(stop_words='english',max_features=50000)

ctmTr = cv.fit_transform(train_clean['text'])

X_test_dtm = cv.transform(test_clean['text'])

In [8]:
#Use TFiDF to optimze for the SDG classifier
tf = TfidfTransformer()

trainTF = tf.fit_transform(ctmTr)
testTF = tf.transform(X_test_dtm)

In [ ]:
#Model, first tried with a normal Support Vecotr Machine, but i takes way too long time...
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto',verbose=True)
SVM.fit(ctmTr,train_clean['label'])

#Predict
predictions_SVM = SVM.predict(X_test_dtm)


[LibSVM]

In [9]:
#Model 2 after research on stackoverflow I decided to use the SGDClaffissier from sklearn, which worked much faster and reached a good result
SGD= SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3,  max_iter=1000, verbose=2,n_iter_no_change=10)
SGD.fit(trainTF,train_clean['label'])  

#Predict 2
predictions_SGD = SGD.predict(testTF)


-- Epoch 1
Norm: 16.83, NNZs: 50000, Bias: -0.074038, T: 3600000, Avg. loss: 0.557095
Total training time: 2.41 seconds.
-- Epoch 2
Norm: 16.82, NNZs: 50000, Bias: -0.073839, T: 7200000, Avg. loss: 0.556588
Total training time: 4.76 seconds.
-- Epoch 3
Norm: 16.82, NNZs: 50000, Bias: -0.073326, T: 10800000, Avg. loss: 0.556672
Total training time: 7.08 seconds.
-- Epoch 4
Norm: 16.82, NNZs: 50000, Bias: -0.073303, T: 14400000, Avg. loss: 0.556686
Total training time: 9.40 seconds.
-- Epoch 5
Norm: 16.82, NNZs: 50000, Bias: -0.072680, T: 18000000, Avg. loss: 0.556653
Total training time: 11.77 seconds.
-- Epoch 6
Norm: 16.82, NNZs: 50000, Bias: -0.073183, T: 21600000, Avg. loss: 0.556651
Total training time: 14.14 seconds.
-- Epoch 7
Norm: 16.82, NNZs: 50000, Bias: -0.073068, T: 25200000, Avg. loss: 0.556638
Total training time: 16.56 seconds.
-- Epoch 8
Norm: 16.82, NNZs: 50000, Bias: -0.073065, T: 28800000, Avg. loss: 0.556642
Total training time: 18.84 seconds.
-- Epoch 9
Norm: 16.82

In [11]:
#Even tho is didint improved through the epochs (i tested with 500 it doesnt change) we get good result of 84% Acc:

print("SVM Accuracy Score -> ",accuracy_score(predictions_SGD, test_clean['label'])*100)

print(classification_report(test_clean['label'], predictions_SGD))


SVM Accuracy Score ->  84.747
              precision    recall  f1-score   support

          1        0.84      0.86      0.85    200000
          2        0.86      0.83      0.84    200000

    accuracy                           0.85    400000
   macro avg       0.85      0.85      0.85    400000
weighted avg       0.85      0.85      0.85    400000



In [ ]:
#Neural Network:
    
#We need vector representations of the sentences. I decided to use keras tokenizer function and a small LSTM Model for this task

In [12]:
#Some Hyperparamaters for the following functions. I decided to count only top 10.000 words because otherwise neural network takes long time for every epoch.
#Also reduced the max sentence lenght to only 100 words and decided for a vector embedding dimension of 100.
max_words = 10000
max_sent_leng=100
embedding_dim=100


In [13]:
#Tokenizer functions to preprocess the data. Fit on Train Data and print the amount of individual tokens.

tokenizer = Tokenizer(num_words=max_words, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True,oov_token = True)
tokenizer.fit_on_texts(train_clean['text'].values)
word_index = tokenizer.word_index
print(len(word_index))

1103004


In [14]:
#From Token to int values of the dictonary plus padding of the sentences that are shorter or longer than 100 words.
#The number 1 stand for words that are not in the 10k words dictionary created above

X_train_nn = tokenizer.texts_to_sequences(train_clean['text'].values)
print(X_train_nn[2])
X_train_nn = pad_sequences(X_train_nn, maxlen=max_sent_leng)
print(X_train_nn.shape)

[351, 9, 1023, 10, 22, 298, 118, 8, 28, 56, 801, 164, 2, 2328, 6139, 8, 1, 8, 4963, 89, 896, 28, 2, 46, 29, 965, 529, 2, 145, 3, 2, 398, 11, 5, 6273, 2431, 3, 599, 71, 6382, 2, 304, 21, 88, 42, 675, 2656, 6, 43, 1301, 875, 22, 2247, 138, 2, 1375, 1387, 532, 33, 1, 1892, 1, 1, 56, 8, 2, 1, 3, 1, 1, 1, 8, 1, 4921, 24, 28, 444, 1362, 23, 68, 9, 1023, 10, 351, 118, 312, 2, 87, 8, 9, 1, 92, 4, 586, 315, 2, 1, 1023, 27, 4, 157, 153, 12, 233, 3, 73, 29, 965, 122, 529, 2, 145, 7, 39, 26, 163, 790, 2, 170, 6, 82, 7, 4, 341, 4, 101, 185, 7, 399, 256]
(3600000, 100)


In [15]:
#Same for the test data
X_test_nn = tokenizer.texts_to_sequences(test_clean['text'].values)
X_test_nn = pad_sequences(X_test_nn, maxlen=max_sent_leng)
print(X_test_nn.shape)
print(X_test_nn[1])

(400000, 100)
[   3    7 2573   25    8   22  298  574   57   10   42  970  913    8
  216 2611    3 1711  159  186  699    3  367  532    4  353   33   23
  508   15   83  111    8  186 2712    8  159   11   22   75  274  145
 6652    4  180 1230   13   25    8    2  159  154    5 6273 2431   44
 1093 2110    6   22  906   19  111 4970   22   25 1311   41    9 1023
   10   13   34   91  697    1  947   11  111    8    2  159   89    4
  137 3760   17   73   29  186 1718  754    4   39  127 1045    2  371
  163    7]


In [16]:
#Still our train sample it huge with 3.6 mio samples. I decided to reduce the sample size by 50% because of computational / time limits

X_train_nn = X_train_nn[:1800000]
train_clean_new = train_clean['label']
train_clean_new = train_clean_new[:1800000]

In [20]:
#Now create the model:

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=X_test_nn.shape[1]))
model.add(LSTM(embedding_dim))
model.add(Dense(1, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 100, 100)          1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,080,501
Trainable params: 1,080,501
Non-trainable params: 0
_________________________________________________________________
None


In [18]:
epochs = 5
batch_size = 258

In [21]:
#Fit the model. Sadly i had only times for 5 Epochs and a small layered LSTM without dropouts. Therefore the model never really fitted 
#and did not well at predicting

history = model.fit(X_train_nn, train_clean_new, epochs=epochs, batch_size=batch_size,validation_split=0.1,verbose=True)

C:\Users\Jonas\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 1620000 samples, validate on 180000 samples
Epoch 1/5
1620000/1620000 [==============================] - 2523s 2ms/step - loss: -7.7042 - accuracy: 0.4975 - val_loss: -7.5797 - val_accuracy: 0.5029
Epoch 2/5
1620000/1620000 [==============================] - 2358s 1ms/step - loss: -7.7042 - accuracy: 0.4975 - val_loss: -7.5797 - val_accuracy: 0.5029
Epoch 3/5
1620000/1620000 [==============================] - 2549s 2ms/step - loss: -7.7042 - accuracy: 0.4975 - val_loss: -7.5797 - val_accuracy: 0.5029
Epoch 4/5
1620000/1620000 [==============================] - 2542s 2ms/step - loss: -7.7042 - accuracy: 0.4975 - val_loss: -7.5797 - val_accuracy: 0.5029
Epoch 5/5
1620000/1620000 [==============================] - 2549s 2ms/step - loss: -7.7042 - accuracy: 0.4975 - val_loss: -7.5797 - val_accuracy: 0.5029


In [22]:
acc = model.evaluate(X_test_nn,test_clean['label'])

400000/400000 [==============================] - 347s 868us/step


In [23]:
#Model didnt really learn, there for random selecting label 1 or 2 and getting only 50% acc. Models needs a better structure and more epochs
#Time and computantional lack to achieve good results
print(acc)


[-7.624619323925972, 0.5]


In [ ]:
#Final Conclution: Fasttest scored the highest Acc with 94% followed by standard TFIDF + SGD classifier with 84% and an not well 
#performing LSTM with 50% acc. Fasttest was also very fast (lol) and worked really well woth its word embedding vectors.